In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader('06 - Todd Dickey - IRSC 2022 (Introduction to Industrial Robot Safety ISO 10218 Parts 1 and 2).pdf')
documents=loader.load()  

In [3]:
# import required software for text splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,   
    chunk_overlap=20,
)

In [5]:
#let us load chunks
mychunks=text_splitter.split_documents(documents) 

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [7]:
import faiss  
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS    #this is used to create actual database

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))) 

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [8]:
vector_store.add_documents(documents=mychunks)

['364be8b7-026e-449c-89a9-7d1ee46e63bb',
 '73b733eb-bba6-4049-a04c-0fd20585bc17',
 'a7da33f4-38d1-4632-b73d-02c89be2ed21',
 '94409405-61a5-4597-8f50-6b2abe5e60fd',
 '511627fa-98b6-41a6-8a1e-961e7a011976',
 '359dd92c-39a9-40f2-86bb-73f6bd58e965',
 '00b42cd9-3a6f-4bcf-8ae6-3bb695602a26',
 '4c43ba9b-3191-4d82-a8c3-12327f26ae9b',
 '2fef12c8-75d8-40e9-b32a-267a5e818838',
 '930d9986-00ed-4afb-9c17-d083d9dd24ba',
 '3c462fd7-100c-44c7-9494-070cec5a1b25',
 '5ccba67d-c828-4607-b444-2886e17f8684',
 'cc09f35b-380e-4b8f-b096-b9aad5892dd3',
 '4d2476ff-5593-4bcd-81b7-b2ac453634c4',
 '642606f3-76d7-4d68-b3cf-c15acf9b66ef',
 'a58dda1b-ac60-4c3a-b493-a9cdd5d0b569',
 'd32f59fb-64b0-49e8-8999-ce4579550157',
 'bf198904-8df0-499e-ac8d-74a853a18dc5',
 '0ad61fd1-787e-426f-92f7-c1c3477b92be',
 '605e5499-8837-4ed0-a8aa-daeefa61c8b8',
 'd419ba85-57ef-4b10-956e-f644bfa9dfe1',
 '84b83cfe-81db-486a-9146-6d02694ce368',
 '4c823636-cb7a-485d-8df6-f581a8f73d82',
 '863b4e20-d6ea-4670-ab90-63066950dd18',
 '038bc989-88b9-

In [23]:
context=vector_store.similarity_search( 'are ISO 10218 and ANSI/RIA R15.06 both same or different?',k=1)
print(context) 
print(type(context[0]))

[Document(id='a0046dda-880a-4896-a620-879a973de352', metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2022-09-28T06:55:41-04:00', 'title': 'PowerPoint Presentation', 'author': 'Robert Huschka', 'moddate': '2022-09-28T06:55:41-04:00', 'source': '06 - Todd Dickey - IRSC 2022 (Introduction to Industrial Robot Safety ISO 10218 Parts 1 and 2).pdf', 'total_pages': 26, 'page': 23, 'page_label': '24'}, page_content='Summary\n● ISO 10218 and ANSI/RIA R15.06 are essentially the same.\n● Standards are created by volunteers from a variety \nprofessional disciplines. \n● Standards provide normative and informative guidance and \ncan be used as a means to achieve regulatory compliance.\n● Functional safety ensures that safety functions relying on a')]
<class 'langchain_core.documents.base.Document'>


In [24]:
actual_context=context[0].page_content
print(actual_context)

Summary
● ISO 10218 and ANSI/RIA R15.06 are essentially the same.
● Standards are created by volunteers from a variety 
professional disciplines. 
● Standards provide normative and informative guidance and 
can be used as a means to achieve regulatory compliance.
● Functional safety ensures that safety functions relying on a


In [29]:
our_question='are ISO 10218 and ANSI/RIA R15.06 both same or different?'

our_prompt=f'''
    i am asking a question , please answer from the given context 
    don't tell the answer if you dont't know.

    context : {actual_context}
    question : {our_question}
'''

In [30]:
import os 
os.environ["OPENAI_API_KEY"] = "enter secrete key"


In [31]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input=our_prompt
)

print(response.output_text)

ISO 10218 and ANSI/RIA R15.06 are essentially the same.
